# 使用MindSpore训练一个简单网络

本节通过MindSpore的API快速实现一个简单的神经网络模型

参考资料：https://www.mindspore.cn/tutorials/zh-CN/r1.9/beginner/quick_start.html#

In [1]:
import mindspore
from mindspore import nn
from mindspore import ops
from mindvision import dataset
from mindspore.dataset import vision
from mindvision.dataset import Mnist

ModuleNotFoundError: No module named 'mindvision'

In [ ]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')
print(train_dataset.get_col_names())

## 处理数据集
MindSpore提供基于Pipeline的数据引擎，通过数据集（Dataset）和数据变换（Transforms）实现高效的数据预处理。此外，MindSpore提供了针对特定领域的开发库，如Text、Vision等。开发库提供大量数据集的封装，能够快速下载使用。在本教程中，我们使用Vision进行数据集的示范操作。

mindvision.dataset模块包括各种CV数据集，如Mnist、Cifar、ImageNet等。在本教程中，我们使用Mnist数据集，自动下载完成后，使用mindspore.dataset提供的数据变换进行预处理。

In [ ]:
# Download training data from open datasets
training_data = dataset.Mnist(
    path="dataset",
    split="train",
    download=True
)

# Download test data from open datasets
test_data = dataset.Mnist(
    path="dataset",
    split="test",
    download=True
)

数据下载完成后，获得数据集对象。

In [ ]:
train_dataset = training_data.dataset
test_dataset = test_data.dataset

打印数据集中包含的数据列名，用于dataset的预处理。

In [ ]:
print(train_dataset.column_names)

这里我们设置batch大小为64，并定义图像所需要做的数据变换，包括Rescale、Normalize、HWC2CHW。

In [ ]:
batch_size = 64

transforms = [
    vision.Rescale(1.0 / 255.0, 0),
    vision.Normalize(mean=(0.1307,), std=(0.3081,)),
    vision.HWC2CHW()
]

MindSpore的dataset使用数据处理流水线（Data Processing Pipeline），需指定map、batch和shuffle等操作。这里我们使用map对'image'列的图像数据进行变换处理，然后将处理好的数据集打包成大小为64的batch。

In [ ]:
# Map vision transforms and batch dataset
train_dataset = train_dataset.map(transforms, 'image').batch(batch_size)
test_dataset = test_dataset.map(transforms, 'image').batch(batch_size)

使用create_tuple_iterator或create_dict_iterator对数据集进行迭代。

In [ ]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break
    


for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break


## 网络构建
mindspore.nn类是构建所有网络的基类，也是网络的基本单元。当用户需要自定义网络时，可以继承nn.Cell类，并重写__init__方法和construct方法。__init__包含所有网络层的定义，construct中包含数据（Tensor）的变换过程（即计算图的构造过程）。

In [ ]:
# Define model
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
print(model)

## 模型训练

In [ ]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

在模型训练中，一个完整的训练过程（step）需要实现以下三步：
1. 正向计算：模型预测结果（logits），并与正确标签（label）求预测损失（loss）。
2. 反向传播：利用自动微分机制，自动求模型参数（parameters）对于loss的梯度（gradients）。
3. 参数优化：将梯度更新到参数上。

MindSpore使用函数式自动微分机制，因此针对上述步骤需要实现：
1. 正向计算函数定义。
2. 通过函数变换获得梯度计算函数。
3. 训练函数定义，执行正向计算、反向传播和参数优化。

In [ ]:

def train(model, dataset, loss_fn, optimizer):
    # Define forward function
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # Get gradient function
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # Define function of one-step training
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")


除训练外，我们定义测试函数，用来评估模型的性能。

In [ ]:

def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


训练过程需多次迭代数据集，一次完整的迭代称为一轮（epoch）。在每一轮，遍历训练集进行训练，结束后使用测试集进行预测。打印每一轮的loss值和预测准确率（Accuracy），可以看到loss在不断下降，Accuracy在不断提高。

In [ ]:

epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)
    test(model, test_dataset, loss_fn)
print("Done!")


##  保存模型
模型训练完成后，需要将其参数进行保存

In [ ]:

# Save checkpoint
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")


## 加载模型
加载保存的权重分为两步：
1. 重新实例化模型对象，构造模型。
2. 加载模型参数，并将其加载至模型上。

In [ ]:

# Instantiate a random initialized model
model = Network()
# Load checkpoint and load parameter to model
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)


加载后的模型可以直接用于预测推理。

In [ ]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break
